In [86]:
import pandas as pd
from sklearn import linear_model as lm
from sklearn.linear_model import Lasso
from cubyc import Run

In [65]:
raptor_23 = pd.read_csv("latest_raptor_by_player.csv")[['player_name','raptor_total','mp']] #from fivethirtyeight github
rookies_23 = pd.read_csv("rookies_23.csv").drop(columns='Rk') #from stathead
college_22 = pd.read_csv("college_22.csv").drop(columns='Rk') #from stathead

In [66]:
def height_to_inches(height_str):
    feet, inches = map(int, height_str.split('-'))
    return feet * 12 + inches

# Modify Data
college_22['Ht_inches'] = college_22['Ht.'].apply(height_to_inches)
college_22['Class'] = college_22['Class'].replace(['FR', 'SO', 'JR', 'SR'], [1, 2, 3, 4])

In [67]:
college_22.columns

Index(['Player', 'Ht.', 'Draft Team', 'Round', 'Pick', 'Draft Year',
       'Draft College', 'Season', 'Team', 'G', 'GS', 'MP', 'FG', 'FGA', '2P',
       '2PA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'FG%', '2P%', '3P%', 'FT%', 'TS%', 'eFG%',
       'Pos', 'Class', '-9999', 'Ht_inches'],
      dtype='object')

In [88]:
features = college_22.merge(raptor_23, left_on='Player', right_on='player_name', how='left').sort_values('Pick')
features = features[features['mp']>500].drop(columns=['Ht.', 'GS', 'Draft Team', 'Round', 'Pick', 'Draft Year',
       'Draft College', 'Season', 'Team','2P', '3P', 'FT', 'TS%', 'eFG%', 'Pos','-9999','player_name','mp'])
features

,Player,G,MP,FG,FGA,2PA,3PA,FTA,ORB,DRB,...,TOV,PF,PTS,FG%,2P%,3P%,FT%,Class,Ht_inches,raptor_total
9,Paolo Banchero,39,33.0,6.3,13.2,9.8,3.3,4.8,1.7,6.1,...,2.4,1.9,17.2,0.478,0.525,0.338,0.729,1,82,-3.088664
16,Keegan Murray,35,31.9,8.8,15.8,11.1,4.7,5.4,2.9,5.8,...,1.1,1.9,23.5,0.554,0.621,0.398,0.747,2,80,-1.292859
37,Jaden Ivey,36,31.4,5.6,12.3,7.3,5.0,5.8,0.7,4.2,...,2.6,1.8,17.3,0.460,0.531,0.358,0.744,2,76,-5.045824
20,Bennedict Mathurin,37,32.5,5.9,13.1,7.0,6.1,4.8,1.4,4.2,...,1.8,1.8,17.7,0.450,0.521,0.369,0.764,2,79,-2.400524
11,Jeremy Sochan,30,25.1,3.3,7.0,4.3,2.7,3.0,2.0,4.4,...,1.6,2.3,9.2,0.474,0.585,0.296,0.589,1,81,-1.946282
26,Jalen Williams,33,34.8,6.6,12.9,9.7,3.2,4.3,0.6,3.8,...,2.1,2.6,18.0,0.513,0.551,0.396,0.809,3,78,-1.579938
4,Jalen Duren,29,25.3,4.9,8.1,8.1,0.0,3.6,3.0,5.2,...,2.2,2.7,12.0,0.597,0.600,0.000,0.625,1,83,-1.904108
35,Ochai Agbaji,39,35.1,6.6,13.9,7.4,6.5,3.9,0.8,4.2,...,2.1,1.7,18.8,0.475,0.534,0.407,0.743,4,77,-2.975622
3,Mark Williams,39,23.6,4.9,6.8,6.8,0.0,2.0,2.6,4.8,...,0.9,2.1,11.2,0.721,0.723,0.000,0.727,2,85,-1.260019
24,AJ Griffin,39,24.0,3.7,7.6,3.5,4.1,1.4,0.8,3.2,...,0.6,1.1,10.4,0.493,0.547,0.447,0.792,1,78,-0.213109


In [89]:
X_train = features.drop(columns=['Player','raptor_total'])
Y_train = features['raptor_total']

In [90]:
draft_model = Lasso(alpha=0.1)
draft_model.fit(X_train, Y_train)
Y_fitted = draft_model.predict(X_train)
Y_fitted

array([-3.91838438, -1.36605658, -4.70731682, -3.51938724, -2.30648263,
       -3.31226529, -2.94692872, -3.39662091,  0.94904772, -2.36448982,
       -2.39208837, -4.54722948, -1.87580262,  2.30665161, -2.65132102,
       -6.55249907, -2.61652947, -2.65886327,  0.56753459, -0.58904998,
       -4.10782823, -5.4247424 , -4.7798825 ])